# Family Facial Recognition for Home Security Applications
By Constantinos Skevofilax and Nikhil Sharma 

The goal of this project is to create a ML model that is able to classify members of a household over 'others', with the goal of integrating this model with a home security camera to determine who is entering a household and notifying the homeowner of who is visiting. 

In [14]:
import os
import random
import numpy as np
from matplotlib import pyplot as plt

In [15]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten
import tensorflow as tf

In [3]:
# http://vis-www.cs.umass.edu/lfw/
# LFW Dataset for the 'strangers' 
# Need to download the folder by going to the above link, clicking on 'Download', and clicking on:
# 'All images as gzipped tar file'
!tar -xf lfw.tgz

In [173]:
# Move the LFW Images to the data/strangers director 
# Move LFW Images to the following repository data/negative
for directory in os.listdir('lfw'):
    for file in os.listdir(os.path.join('lfw', directory)):
        EX_PATH = os.path.join('lfw', directory, file)
        NEW_PATH = os.path.join('data/strangers', file)
        os.replace(EX_PATH, NEW_PATH)

## Data Augmentation 
We want to create a comparable family data set that will match the size of the negatives dataset so we can have a roughly equal in size to compare against. We will achieve this by using image augmentation methods such as affine transformation, random color jitter, and affine transformation with rotation. 

In [16]:
# Installing necessary packages to augment the images 
import urllib.request
import shutil
from IPython.display import Image

In [5]:
pip install torchvision


[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### Image Augmentation

In [17]:
import os
import torch
import torchvision.transforms as transforms
from torchvision.utils import make_grid
from PIL import Image

In [18]:
from torchvision.transforms.functional import to_pil_image

# We want to save the augmentations to their respective subdirectories 
def save_augmented_images(img_file, transform, save_path):
    # Load the requested image
    img = Image.open(img_file).resize((256, 256))

    # Apply transformations to the image
    augmented_images = [transform(img) for _ in range(4)]
    for i, augmented_img_tensor in enumerate(augmented_images):
        # Convert tensor to PIL Image
        augmented_img_pil = to_pil_image(augmented_img_tensor)
        # Save augmented image
        augmented_img_pil.save(os.path.join(save_path, f"{os.path.splitext(os.path.basename(img_file))[0]}_aug_{i}.jpg"))


In [19]:
# Flips, "moves", adjusts brightness, and rotates family dataset to have a comparable number to the negatives 
# dataset size (1555 images)
transforms_list = [
    transforms.Compose([
        transforms.RandomAffine(degrees=0, translate=(0.5, 0)),
        transforms.Pad(padding=200, fill=0, padding_mode='edge'),
        transforms.ToTensor()
    ]),
    transforms.Compose([
        transforms.ColorJitter(brightness=(0, 1)),
        transforms.ToTensor()
    ]),
    transforms.Compose([
        transforms.RandomAffine(degrees=30),
        transforms.ToTensor()
    ])
]

In [100]:
# Set the data path
DATA_PATH = '/root/Final_Project/data/'

In [102]:
# Query all the family sub directories that we want to augment (exclude the strangers because there's too many) 
subdirectories = [
    os.path.join(DATA_PATH, sub_dir) 
    for sub_dir in os.listdir(DATA_PATH) 
    if os.path.isdir(os.path.join(DATA_PATH, sub_dir)) and sub_dir != 'strangers'
]
subdirectories

['/root/Final_Project/data/athena',
 '/root/Final_Project/data/costaki',
 '/root/Final_Project/data/george',
 '/root/Final_Project/data/teresa']

In [24]:
# For each of the family member sub directories, run through the transformations and save the augmentations
for subdirectory in subdirectories:
    for img_file in os.listdir(subdirectory):
        if img_file.endswith('.jpg'):
            img_path = os.path.join(subdirectory, img_file)
            for transform in transforms_list:
                save_augmented_images(img_path, transform, subdirectory)

## Convolutional Neural Network

In [91]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [112]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

base_dir = '/root/Final_Project/data/'

train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.3)  # rescale pixel values and split data

train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(250, 250),
    batch_size=32,
    class_mode='categorical',  # Automatically label the images based on subdirectory names
    subset='training')

validation_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(250, 250),
    batch_size=32,
    class_mode='categorical',  # Automatically label the images based on subdirectory names
    subset='validation')


Found 3640 images belonging to 5 classes.
Found 1557 images belonging to 5 classes.


In [113]:
print("Class indices:", train_generator.class_indices)

Class indices: {'athena': 0, 'costaki': 1, 'george': 2, 'strangers': 3, 'teresa': 4}


In [114]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, AveragePooling2D, MaxPooling2D, Flatten, Dense, Dropout

model_CNN = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(250, 250, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(5, activation='softmax')  # Assuming 4 family members + others
])

model_CNN.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [158]:
model_CNN.summary()

Model: "sequential_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_30 (Conv2D)              │ (None, 248, 248, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_27 (MaxPooling2D) │ (None, 124, 124, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_31 (Conv2D)              │ (None, 122, 122, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_28 (MaxPooling2D) │ (None, 61, 61, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_32 (Conv2D)              │ (None, 59, 59, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_29 (MaxPooling2D) │ (None, 29, 29, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_10 (Flatten)            │ (None, 107648)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ (None, 512)            │    55,116,288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 5)              │         2,565 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 55,212,103 (210.62 MB)

 Trainable params: 55,212,101 (210.62 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [115]:
history = model_CNN.fit(
    train_generator,
    epochs=5,
    validation_data=validation_generator)

Epoch 1/5
114/114 ━━━━━━━━━━━━━━━━━━━━ 132s 1s/step - accuracy: 0.5000 - loss: 1.4721 - val_accuracy: 0.4560 - val_loss: 1.6477
Epoch 2/5
114/114 ━━━━━━━━━━━━━━━━━━━━ 130s 1s/step - accuracy: 0.9379 - loss: 0.1864 - val_accuracy: 0.5260 - val_loss: 2.3739
Epoch 3/5
114/114 ━━━━━━━━━━━━━━━━━━━━ 130s 1s/step - accuracy: 0.9761 - loss: 0.0672 - val_accuracy: 0.5414 - val_loss: 2.1742
Epoch 4/5
114/114 ━━━━━━━━━━━━━━━━━━━━ 131s 1s/step - accuracy: 0.9854 - loss: 0.0436 - val_accuracy: 0.5581 - val_loss: 2.7173
Epoch 5/5
114/114 ━━━━━━━━━━━━━━━━━━━━ 128s 1s/step - accuracy: 0.9927 - loss: 0.0207 - val_accuracy: 0.5498 - val_loss: 2.6626


In [162]:
# We apply testing of the accuracy of our model by presenting new testing data
# These images the model has not seen before, both of the family and of the strangers set 
# The following are the class indicies that the model sees:
# Class indices: {'athena': 0, 'costaki': 1, 'george': 2, 'strangers': 3, 'teresa': 4}

from keras.preprocessing import image

val_loss, val_acc = model.evaluate(validation_generator)
print(f"Validation accuracy: {val_acc}")

#model_CNN.save('family_member_classifier_CNN.h5') #<- Uncomment as needed

# Load and use the model
from tensorflow.keras.models import load_model
model_CNN = load_model('family_member_classifier_CNN.h5')

import numpy as np
#img = image.load_img('test_images/athena_test_0004.jpg', target_size=(250, 250)) 
#img = image.load_img('test_images/costaki_test_0001.jpg', target_size=(250, 250))
#img = image.load_img('test_images/george_test_0004.jpg', target_size=(250, 250))
#img = image.load_img('test_images/teresa_test_0003.jpg', target_size=(250, 250))
#img = image.load_img('test_images/Johnny_Cash.jpg', target_size=(250, 250))

img_tensor = image.img_to_array(img)
img_tensor = np.expand_dims(img_tensor, axis=0)
img_tensor /= 255.

prediction = model_CNN.predict(img_tensor)
print(prediction)



49/49 ━━━━━━━━━━━━━━━━━━━━ 11s 230ms/step - accuracy: 0.1860 - loss: 1.6143
Validation accuracy: 0.19781631231307983


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
[[7.2531941e-05 1.2819569e-03 1.6549295e-04 9.9733812e-01 1.1418393e-03]]


### CNN Lenet-5
We opted for the Lenet-5 model because of usage in image classification. Based on our prior research, we believed that the Lenet-5 model would be one of the best performing models in differentating between family members and strangers. We opted for 5 layers (including the output), with 6 filters.  

In [120]:
model_lenet5 = Sequential()

# Layer 1: Convolutional layer with 6 filters of size 3x3, followed by average pooling
model_lenet5.add(Conv2D(6, kernel_size=(3, 3), activation='relu', input_shape=(250,250,3)))
model_lenet5.add(AveragePooling2D(pool_size=(2, 2)))

# Layer 2: Convolutional layer with 16 filters of size 3x3, followed by average pooling
model_lenet5.add(Conv2D(16, kernel_size=(3, 3), activation='relu'))
model_lenet5.add(AveragePooling2D(pool_size=(2, 2)))

# Flatten the feature maps to feed into fully connected layers
model_lenet5.add(Flatten())

# Layer 3: Fully connected layer with 120 neurons
model_lenet5.add(Dense(120, activation='relu'))

# Layer 4: Fully connected layer with 84 neurons
model_lenet5.add(Dense(84, activation='relu'))

# Output layer: Fully connected layer with 1 neuron
model_lenet5.add(Dense(5, activation='softmax'))

# Compile model
model_lenet5.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# Generating the summary of the model
model_lenet5.summary()

Model: "sequential_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_33 (Conv2D)              │ (None, 248, 248, 6)    │           168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_2             │ (None, 124, 124, 6)    │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_34 (Conv2D)              │ (None, 122, 122, 16)   │           880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_3             │ (None, 61, 61, 16)     │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_11 (Flatten)            │ (None, 59536)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 120)            │     7,144,440 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_24 (Dense)                │ (None, 84)             │        10,164 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_25 (Dense)                │ (None, 5)              │           425 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,156,077 (27.30 MB)

 Trainable params: 7,156,077 (27.30 MB)

 Non-trainable params: 0 (0.00 B)

In [121]:
history = model_lenet5.fit(
    train_generator,
    epochs=5,
    validation_data=validation_generator)

Epoch 1/5
114/114 ━━━━━━━━━━━━━━━━━━━━ 35s 292ms/step - accuracy: 0.6083 - loss: 1.0649 - val_accuracy: 0.4277 - val_loss: 2.5603
Epoch 2/5
114/114 ━━━━━━━━━━━━━━━━━━━━ 40s 288ms/step - accuracy: 0.9695 - loss: 0.1146 - val_accuracy: 0.4727 - val_loss: 3.2602
Epoch 3/5
114/114 ━━━━━━━━━━━━━━━━━━━━ 33s 290ms/step - accuracy: 0.9956 - loss: 0.0208 - val_accuracy: 0.5517 - val_loss: 3.2294
Epoch 4/5
114/114 ━━━━━━━━━━━━━━━━━━━━ 33s 288ms/step - accuracy: 0.9938 - loss: 0.0251 - val_accuracy: 0.5459 - val_loss: 3.8295
Epoch 5/5
114/114 ━━━━━━━━━━━━━━━━━━━━ 33s 290ms/step - accuracy: 0.9974 - loss: 0.0122 - val_accuracy: 0.4836 - val_loss: 3.9203


In [172]:
# We apply testing of the accuracy of our model by presenting new testing data
# These images the model has not seen before, both of the family and of the strangers set 
# The following are the class indicies that the model sees:
# Class indices: {'athena': 0, 'costaki': 1, 'george': 2, 'strangers': 3, 'teresa': 4}

from keras.preprocessing import image

val_loss, val_acc = model.evaluate(validation_generator)
print(f"Validation accuracy: {val_acc}")

#model_lenet5.save('family_member_classifier_lenet5.h5') #<- Uncomment as needed

# Load and use the model
from tensorflow.keras.models import load_model
model_lenet5 = load_model('family_member_classifier_lenet5.h5')

import numpy as np
img = image.load_img('test_images/athena_test_0007.jpg', target_size=(250, 250))  
#img = image.load_img('test_images/costaki_test_0002.jpg', target_size=(250, 250)) 
#img = image.load_img('test_images/george_test_0003.jpg', target_size=(250, 250))
#img = image.load_img('test_images/teresa_test_0003.jpg', target_size=(250, 250))
#img = image.load_img('test_images/marilyn_monroe.jpg', target_size=(250, 250))

img_tensor = image.img_to_array(img)
img_tensor = np.expand_dims(img_tensor, axis=0)
img_tensor /= 255.

prediction = model_lenet5.predict(img_tensor)
print(prediction)



49/49 ━━━━━━━━━━━━━━━━━━━━ 11s 229ms/step - accuracy: 0.2056 - loss: 1.6140
Validation accuracy: 0.19781631231307983


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
[[6.8331152e-02 1.5640017e-02 7.0141692e-07 9.0729171e-01 8.7364623e-03]]
